In [ ]:
from sklearn.metrics import (accuracy_score, f1_score,
                             precision_score, recall_score)
from tqdm.auto import tqdm


from viterbi_utils import *

In [ ]:
TRAIN_PATH = r"./data/train.txt"
TEST_PATH = r"./data/test.txt"

train_loader = Loader(TRAIN_PATH)
test_loader = Loader(TEST_PATH)
calculate_all_counts(train_loader)
print(all_states)

In [ ]:
def viterbi(sentence: Sentence) -> List[str]:
    # Get the words from the sentence
    words = [word.word for word in sentence]
    # words = [word.norm_word for word in sentence]

    # Number of words in the sentence
    n = len(sentence)

    # Create a table to store the best scores and back pointers
    # Each cell stores a dictionary that maps a state to the best score and back pointer
    # [word_index] -> {state: (score, back_pointer)}
    table: List[Dict[str, Tuple[float, Optional[str]]]] = [{}
                                                           for _ in range(n)]

    # Initialize the first column of the table
    for state in all_states:
        table[0][state] = (get_emission_probability(words[0], state) +
                           get_start_probability(state),
                           None)
    # END for state in all_states

    # Fill in the rest of the table
    for i in range(1, n):
        for state in all_states:
            best_score = float('inf')
            back_pointer = None

            for prev_state in all_states:
                score = (table[i - 1][prev_state][0] +
                         get_transition_probability(prev_state, state) +
                         get_emission_probability(words[i], state))

                # print(f"{words[i] = :10} {score = :0.5f} {best_score = :0.5f} "
                #       f"{prev_state = :5} {state = :5} {back_pointer = }")

                if score < best_score:  # less than because -ve log probs
                    best_score = score
                    back_pointer = prev_state
                # END if score < best_score
            # END for prev_state in all_states

            table[i][state] = (best_score, back_pointer)
        # END for state in all_states
    # END for i in range(1, n)

    # pp(table)

    # Find the best last state
    best_last_state = min(table[-1], key=lambda k: table[-1][k][0])

    # print(f"{best_last_state = }")

    # Follow the back pointers to find the best path
    best_path: List[str] = []
    back_pointer = best_last_state
    i = n - 1

    while back_pointer is not None and i >= 0:
        best_path.append(back_pointer)
        back_pointer = table[i][back_pointer][1]
        i -= 1

    # Reverse the best path
    best_path.reverse()

    return best_path
# END viterbi

In [ ]:
def save_predictions(
    loader: Loader,
    predictions: List[List[str]],
    filename: str
) -> None:
    with open(filename, "w", encoding="utf-8") as file:
        for sentence, tags in zip(loader, predictions):
            for word, tag in zip(sentence, tags):
                file.write(f"{sentence.sent_id}\t"
                           f"{word.word_index}\t"
                           f"{word.word}\t"
                           f"{tag}\t"
                           f"{word.pos_tag}\n")
            # END for word, tag in zip(sentence, tags)
        # END for sentence, tags in zip(sentences, predictions)

In [ ]:
sentence = train_loader[500]
pred = viterbi(sentence)
actual = [word.pos_tag for word in sentence]

print(sentence.text)
print(pred, len(pred))
print(actual, len(actual))

In [ ]:
# Test on train data
predicted_tags: List[List[str]] = []

print("Train data:")
for sentence in tqdm(train_loader, desc="Train data"):
    predicted_tags.append(viterbi(sentence))
# END for sentence in train_loader

save_predictions(train_loader, predicted_tags, r"viterbi_predictions_train.tsv")

_predicted_tags = sum(predicted_tags, [])
actual_tags = sum(([word.pos_tag for word in sentence]
                   for sentence in train_loader), [])

metrics = {
    "accuracy": accuracy_score(actual_tags,
                               _predicted_tags),
    "recall": recall_score(actual_tags,
                           _predicted_tags,
                           average="macro",
                           zero_division=0),
    "precision": precision_score(actual_tags,
                                 _predicted_tags,
                                 average="macro",
                                 zero_division=0),
    "f1": f1_score(actual_tags,
                   _predicted_tags,
                   average="macro",
                   zero_division=0)
}

pp(metrics)

In [ ]:
# Test on test data
predicted_tags: List[List[str]] = []

print("Test data:")
for sentence in tqdm(test_loader, desc="Test data"):
    predicted_tags.append(viterbi(sentence))
# END for sentence in test_loader

save_predictions(test_loader, predicted_tags, r"viterbi_predictions_test.tsv")

_predicted_tags = sum(predicted_tags, [])
actual_tags = sum(([word.pos_tag for word in sentence]
                   for sentence in test_loader), [])

metrics = {
    "accuracy": accuracy_score(actual_tags,
                               _predicted_tags),
    "recall": recall_score(actual_tags,
                           _predicted_tags,
                           average="macro",
                           zero_division=0),
    "precision": precision_score(actual_tags,
                                 _predicted_tags,
                                 average="macro",
                                 zero_division=0),
    "f1": f1_score(actual_tags,
                   _predicted_tags,
                   average="macro",
                   zero_division=0)
}

pp(metrics)